<a href="https://colab.research.google.com/github/joyceee916/ATMS-597-SP-2020-Project-5/blob/master/Project5_Part1_DataCleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 5 - Classification (version 1.0)
Tasks: Use supervised classification techniques to predict frozen vs. liquid precipitation type (i.e., when it is not raining, so rain and freezing rain vs. other categories such as snow, sleet, etc.) at the surface in ASOS weather observations.

Model 1: Logistic Regression classification with default values (baseline)

>>> from sklearn.linear_model import LogisticRegression
...
>>> clf = LogisticRegression(random_state=0).fit(X, y)
Model 2: Your pick!

Data: You will use 5-minute Automated Surface Observation System data available from NCDC (ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/). Data from 2000-2020 is available. Use a 70-30 random split to perform training and validation.

You can use Temperature, Dewpoint, Wind Direction, Wind Speed, and other derived variables from those data, or synthetic variables such as solar zenith angle, time of day, etc.

Each group will have a separate Midwest US city to examine:

Group D: KGRB (WI)

The group with the improvement of values of Briar Skill Score for their Model 2 over the baseline SVM model will be given a prize.

Course presentations will be Thursday, April 23. A colab notebook or jupyter notebook can be submitted for the assignment.


In [0]:
pwd

'/content/drive/My Drive/ATMS 597/Project5'

In [0]:
cd /content/drive/My Drive/ATMS 597/Project5

/content/drive/My Drive/ATMS 597/Project5


In [0]:
!pip install metpy
!pip install metar

     |████████████████████████████████| 327kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 194kB 8.8MB/s 
     |████████████████████████████████| 215kB 2.7MB/s 
  Created wheel for metar: filename=metar-1.7.0-cp36-none-any.whl size=198905 sha256=0ba4919f0c0eb3a37627e16fcfe510129c52ff4e8b72d04e408bd5a5a3deea4f
  Stored in directory: /root/.cache/pip/wheels/8e/7b/5c/abc5749d1a4d2241b9dc824f6723f699f6eb2c8e6deeb1c294
Successfully built metar


In [0]:
from ftplib import FTP 
import pandas as pd
import metpy
import metpy.io

Cannot import USCOUNTIES and USSTATES without Cartopy installed.


Downloading the Data

In [0]:
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login(user='ftp', passwd='joycey2@illinois.gov') # substitute with your own email

'230-****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n ** This is a United States Department of Commerce computer     **\n ** system, which may be accessed and used only for             **\n ** official Government business by authorized personnel.       **\n ** Unauthorized access or use of this computer system may      **\n ** subject violators to criminal, civil, and/or administrative **\n ** action.  All information on this computer system may be     **\n ** intercepted, recorded, read, copied, and disclosed by and   **\n ** to authorized personnel for official purposes, including    **\n ** criminal investigations.  Access or use of this computer    **\n ** system by any person, whether authorized or unauthorized,   **\n ** constitutes consent to these terms.                         **\n ****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n230 Anonymous access granted, restrictions apply'

In [0]:
# File Name Generator 
def folderNames(year):
    folder_prefix = '/pub/data/asos-fivemin/6401-'
    folder = folder_prefix + str(year) + '/'

    file_prefix = '64010KGRB'
    file_suffix = '.dat'
    file_month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    files = [file_prefix+str(year)+month+file_suffix for month in file_month]

    return folder, files

In [0]:
def grabFile():
    #ftp.cwd('/pub/data/asos-fivemin/6401-2000/')

    #filename = '64010KGRB200001.dat'
    for year in range(2000, 2021):
        folder, files = folderNames(year)
        print(folder)
        try:
            ftp = FTP('ftp.ncdc.noaa.gov')
            ftp.login(user='ftp', passwd='joycey2@illinois.gov')
            ftp.cwd(folder)
        except: 
            print('couldnt log in or change to correct directory')
        for file in files:
            try: 
                localfile = open(file, 'wb')
                ftp.retrbinary('RETR ' + file, localfile.write, 1024)
                localfile.close()
            except: 
                return

In [0]:
#grabFile() 
# importing the data

/pub/data/asos-fivemin/6401-2000/
/pub/data/asos-fivemin/6401-2001/
/pub/data/asos-fivemin/6401-2002/
/pub/data/asos-fivemin/6401-2003/
/pub/data/asos-fivemin/6401-2004/
/pub/data/asos-fivemin/6401-2005/
/pub/data/asos-fivemin/6401-2006/
/pub/data/asos-fivemin/6401-2007/
/pub/data/asos-fivemin/6401-2008/
/pub/data/asos-fivemin/6401-2009/
/pub/data/asos-fivemin/6401-2010/
/pub/data/asos-fivemin/6401-2011/
/pub/data/asos-fivemin/6401-2012/
/pub/data/asos-fivemin/6401-2013/
/pub/data/asos-fivemin/6401-2014/
/pub/data/asos-fivemin/6401-2015/
/pub/data/asos-fivemin/6401-2016/
/pub/data/asos-fivemin/6401-2017/
/pub/data/asos-fivemin/6401-2018/
/pub/data/asos-fivemin/6401-2019/
/pub/data/asos-fivemin/6401-2020/


Parsing the Data

In [0]:
# Courtesy of Max Grover 
from datetime import datetime
import pandas as pd
from metpy.io import parse_metar_to_dataframe
import glob


def parse_metar_file(file, wx_subset=True):
    """
    Parses METAR file from NCDC
    Input:
    --------
    file = Text file downloaded from NCDC
    wx_subset = Flag to determine whether or not to drop non-current weather obs (if True, only returns obs with observed weather)
    Output:
    --------
    df = Pandas dataframe filtered for times where current weather is not 'nan'
    """

    # Read in the file using Pandas
    df = pd.read_csv(file, header=None)

    # Pull the timestamp from the filename
    timestamp = datetime.strptime(file[-10:], '%Y%m.dat')

    # Iterrate over rows to parse METARS
    df_list = []
    for index, row in df.iterrows():
        try:
            df_list.append(parse_metar_to_dataframe(row.values[0][52:], year=timestamp.year, month=timestamp.month))
        except:
            print('Error with METAR: ', row.values[0][52:])
    #
    merged_df = pd.concat(df_list)

    # Drop datasets that do not include current weather
    merged_df = merged_df.dropna(subset=['current_wx1'])

    # Change the index to datetime
    merged_df.index = merged_df.date_time

    # Return the merged dataset sorted by datetime
    return merged_df.sort_index()

In [0]:
# Function that takes in the name of a metar file, parses it into a dataframe, and retains only the rows with precipitation 
def PrecipRows(metar_file):
    df = parse_metar_file(metar_file)
    df_precip = df[df.current_wx1.str.contains("SN|RA")]
    return df_precip

2000

In [0]:
all_files2000 = [] 
for year in range(2000, 2001):
    folder, files = folderNames(year)
    all_files2000.append(files) # creates a list of lists 

for x in all_files2000:
    del x[0]
print(all_files2000)

[['64010KGRB200002.dat', '64010KGRB200003.dat', '64010KGRB200004.dat', '64010KGRB200005.dat', '64010KGRB200006.dat', '64010KGRB200007.dat', '64010KGRB200008.dat', '64010KGRB200009.dat', '64010KGRB200010.dat', '64010KGRB200011.dat', '64010KGRB200012.dat']]


In [0]:
df2000 = PrecipRows('64010KGRB200001.dat')
for list in all_files2000:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2000 = df2000.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2000.head())
print(df2000.current_wx1.unique())
print(len(df2000))

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...       7.517541        2.736161
1       KGRB     44.48  ...       7.878462        1.389185
2       KGRB     44.48  ...       8.863270        1.562834
3       KGRB     44.48  ...       6.577848        2.394141
4       KGRB     44.48  ...       7.517541        2.736161

[5 rows x 28 columns]
['-SN' '-RA' 'RA' 'SN' '-FZRA' '-TSRA' '+RA' '+TSRA' '+SN' 'TSRA']
9746


In [0]:
df2000.to_csv('df2000.csv', index=False)

2001

In [0]:
all_files2001 = [] 
for year in range(2001, 2002):
    folder, files = folderNames(year)
    all_files2001.append(files) # creates a list of lists 

for x in all_files2001:
    del x[0]
print(all_files2001)

[['64010KGRB200102.dat', '64010KGRB200103.dat', '64010KGRB200104.dat', '64010KGRB200105.dat', '64010KGRB200106.dat', '64010KGRB200107.dat', '64010KGRB200108.dat', '64010KGRB200109.dat', '64010KGRB200110.dat', '64010KGRB200111.dat', '64010KGRB200112.dat']]


In [0]:
df2001 = PrecipRows('64010KGRB200101.dat')
for list in all_files2001:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2001 = df2001.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2001.head())
print(df2001.current_wx1.unique())
print(len(df2001))

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...            5.5        9.526279
1       KGRB     44.48  ...            5.0        8.660254
2       KGRB     44.48  ...            5.0        8.660254
3       KGRB     44.48  ...            4.5        7.794229
4       KGRB     44.48  ...            5.5        9.526279

[5 rows x 28 columns]
['-SN' '-FZRA' '-RA' 'SN' '+SN' 'RA' '+RA' 'FZRA' '-TSRA' 'TSRA' '+TSRA']
7966


In [0]:
df2001.to_csv('df2001.csv', index=False)

2002

In [0]:
all_files2002 = [] 
for year in range(2002, 2003):
    folder, files = folderNames(year)
    all_files2002.append(files) # creates a list of lists 

for x in all_files2002:
    del x[0]
print(all_files2002)

[['64010KGRB200202.dat', '64010KGRB200203.dat', '64010KGRB200204.dat', '64010KGRB200205.dat', '64010KGRB200206.dat', '64010KGRB200207.dat', '64010KGRB200208.dat', '64010KGRB200209.dat', '64010KGRB200210.dat', '64010KGRB200211.dat', '64010KGRB200212.dat']]


In [0]:
df2002 = PrecipRows('64010KGRB200201.dat')
for list in all_files2002:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2002 = df2002.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2002.head())
print(df2002.current_wx1.unique())
print(len(df2002))

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...     -12.212965      -14.554844
1       KGRB     44.48  ...     -12.212965      -14.554844
2       KGRB     44.48  ...     -12.855752      -15.320889
3       KGRB     44.48  ...     -14.141327      -16.852978
4       KGRB     44.48  ...     -13.498540      -16.086933

[5 rows x 28 columns]
['-SN' 'SN' '-RA' '-FZRA' 'RA' '+RA' '+SN' 'FZRA' 'TSRA' '+TSRA' '-TSRA']
8158


In [0]:
df2002.to_csv('df2002.csv', index=False)

2003

In [0]:
all_files2003 = [] 
for year in range(2003, 2004):
    folder, files = folderNames(year)
    all_files2003.append(files) # creates a list of lists 

for x in all_files2003:
    del x[0]
print(all_files2003)

[['64010KGRB200302.dat', '64010KGRB200303.dat', '64010KGRB200304.dat', '64010KGRB200305.dat', '64010KGRB200306.dat', '64010KGRB200307.dat', '64010KGRB200308.dat', '64010KGRB200309.dat', '64010KGRB200310.dat', '64010KGRB200311.dat', '64010KGRB200312.dat']]


In [0]:
df2003 = PrecipRows('64010KGRB200301.dat')
for list in all_files2003:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2003 = df2003.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2003.head())
print(df2003.current_wx1.unique())
print(len(df2003))

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...      -3.420201       -9.396926
1       KGRB     44.48  ...      -3.762222      -10.336619
2       KGRB     44.48  ...      -3.420201       -9.396926
3       KGRB     44.48  ...      -3.762222      -10.336619
4       KGRB     44.48  ...      -1.736482       -9.848078

[5 rows x 28 columns]
['-SN' 'SN' '-RA' '-FZRA' '+SN' '+RA' 'RA' 'FZRA' 'TSRA' '+TSRA' '-TSRA']
7674


In [0]:
df2003.to_csv('df2003.csv', index=False)

2004

In [0]:
all_files2004 = [] 
for year in range(2004, 2005):
    folder, files = folderNames(year)
    all_files2004.append(files) # creates a list of lists 

for x in all_files2004:
    del x[0]
print(all_files2004)

[['64010KGRB200402.dat', '64010KGRB200403.dat', '64010KGRB200404.dat', '64010KGRB200405.dat', '64010KGRB200406.dat', '64010KGRB200407.dat', '64010KGRB200408.dat', '64010KGRB200409.dat', '64010KGRB200410.dat', '64010KGRB200411.dat', '64010KGRB200412.dat']]


In [0]:
df2004 = PrecipRows('64010KGRB200401.dat')
for list in all_files2004:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2004 = df2004.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2004.head())
print(df2004.current_wx1.unique())
print(len(df2004))

df2004.to_csv('df2004.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


Error with METAR:   KGRB 029 9540 68 -1500 320/07 RMK AO2
Error with METAR:   KGRB 0288
Error with METAR:   P0000
Error with METAR:   KGRB 03SM CLR M04/M08 A2991 700 74  -1400 320/09 RMK AO2
Error with METAR:   KGRB 038
Error with METAR:  0 TO 34006KT 10SM CLR M08/M11 A3001 620 77 -2000 340/06 RMK AO2 SLP171 T10781111 51027
Error with METAR:   KGRB 033  KGRB 031040Z AUTO 24004KT 10SM CLR M09/M12 A3003 590 80 -2100 240/04 RMK AO2
Error with METAR:   KGRB 03580
Error with METAR:   KGRB 03580 1355Z 00000KT 10SM OVC110 M08/M11 A3003 590  80 -2100 000/00 RMK AO2
Error with METAR:   68  -1700 170/08 RMK AO2
Error with METAR:   KGRB 03 66
Error with METAR:   KGRB 03  71 -1100 190/09 RMK AO2
Error with METAR:  00 81-700 220/12 RMK AO2
Error with METAR:   75 -100 210/13 RMK AO2
Error with METAR:  90 012KT 10SM BKN050 06/M01 A2972 880 62 0 290/12 RMK AO2 $
Error with METAR:  72 $
Error with METAR:   KGRB 05
Error with METAR:  570 81 -1200 030/07 RMK AO2 $
Error with METAR:   KGRB 05SM OVC027 M02

2005

In [0]:
all_files2005 = [] 
for year in range(2005, 2006):
    folder, files = folderNames(year)
    all_files2005.append(files) # creates a list of lists 

for x in all_files2005:
    del x[0]
print(all_files2005)

In [0]:
df2005 = PrecipRows('64010KGRB200501.dat')
for list in all_files2005:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2005 = df2005.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2005.head())
print(df2005.current_wx1.unique())
print(len(df2005))

df2005.to_csv('df2005.csv', index=False)

2006

In [0]:
all_files2006 = [] 
for year in range(2006, 2007):
    folder, files = folderNames(year)
    all_files2006.append(files) # creates a list of lists 

for x in all_files2006:
    del x[0]
print(all_files2006)

[['64010KGRB200602.dat', '64010KGRB200603.dat', '64010KGRB200604.dat', '64010KGRB200605.dat', '64010KGRB200606.dat', '64010KGRB200607.dat', '64010KGRB200608.dat', '64010KGRB200609.dat', '64010KGRB200610.dat', '64010KGRB200611.dat', '64010KGRB200612.dat']]


In [0]:
df2006 = PrecipRows('64010KGRB200601.dat')
for list in all_files2006:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2006 = df2006.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2006.head())
print(df2006.current_wx1.unique())
print(len(df2006))

df2006.to_csv('df2006.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...      -8.660254       -5.000000
1       KGRB     44.48  ...      -6.928203       -4.000000
2       KGRB     44.48  ...      -9.192533       -7.713451
3       KGRB     44.48  ...      -7.713451       -9.192533
4       KGRB     44.48  ...      -8.426489       -7.070664

[5 rows x 28 columns]
['-RA' '-SN' 'RA' '-FZRA' 'SN' '+RA' '+SN' '-TSRA' 'TSRA' '+TSRA']
7115


2007

In [0]:
all_files2007 = [] 
for year in range(2007, 2008):
    folder, files = folderNames(year)
    all_files2007.append(files) # creates a list of lists 

for x in all_files2007:
    del x[0]
print(all_files2007)

In [0]:
df2007 = PrecipRows('64010KGRB200701.dat')
for list in all_files2007:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2007 = df2007.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2007.head())
print(df2007.current_wx1.unique())
print(len(df2007))

df2007.to_csv('df2007.csv', index=False)

2008

In [0]:
all_files2008 = [] 
for year in range(2008, 2009):
    folder, files = folderNames(year)
    all_files2008.append(files) # creates a list of lists 

for x in all_files2008:
    del x[0]
print(all_files2008)

[['64010KGRB200802.dat', '64010KGRB200803.dat', '64010KGRB200804.dat', '64010KGRB200805.dat', '64010KGRB200806.dat', '64010KGRB200807.dat', '64010KGRB200808.dat', '64010KGRB200809.dat', '64010KGRB200810.dat', '64010KGRB200811.dat', '64010KGRB200812.dat']]


In [0]:
df2008 = PrecipRows('64010KGRB200801.dat')
for list in all_files2008:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2008 = df2008.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2008.head())
print(df2008.current_wx1.unique())
print(len(df2008))

df2008.to_csv('df2008.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...      -3.856726       -4.596267
1       KGRB     44.48  ...      -3.856726       -4.596267
2       KGRB     44.48  ...      -5.362311       -4.499513
3       KGRB     44.48  ...      -5.362311       -4.499513
4       KGRB     44.48  ...      -5.362311       -4.499513

[5 rows x 28 columns]
['-SN' '-RA' '-TSRA' '+TSRA' 'TSRA' 'RA' '+RA' 'SN' '+SN' '-FZRA' 'FZRA']
9194


2009

In [0]:
all_files2009 = [] 
for year in range(2009, 2010):
    folder, files = folderNames(year)
    all_files2009.append(files) # creates a list of lists 

for x in all_files2009:
    del x[0]
print(all_files2009)

In [0]:
df2009 = PrecipRows('64010KGRB200901.dat')
for list in all_files2009:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2009 = df2009.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2009.head())
print(df2009.current_wx1.unique())
print(len(df2009))

df2009.to_csv('df2009.csv', index=False)

2010

In [0]:
all_files2010 = [] 
for year in range(2010, 2011):
    folder, files = folderNames(year)
    all_files2010.append(files) # creates a list of lists 

for x in all_files2010:
    del x[0]
print(all_files2010)

[['64010KGRB201002.dat', '64010KGRB201003.dat', '64010KGRB201004.dat', '64010KGRB201005.dat', '64010KGRB201006.dat', '64010KGRB201007.dat', '64010KGRB201008.dat', '64010KGRB201009.dat', '64010KGRB201010.dat', '64010KGRB201011.dat', '64010KGRB201012.dat']]


In [0]:
df2010 = PrecipRows('64010KGRB201001.dat')
for list in all_files2010:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2010 = df2010.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2010.head())
print(df2010.current_wx1.unique())
print(len(df2010))

df2010.to_csv('df2010.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...       4.499513       -5.362311
1       KGRB     44.48  ...       5.785088       -6.894400
2       KGRB     44.48  ...      -2.736161       -7.517541
3       KGRB     44.48  ...      -3.420201       -9.396926
4       KGRB     44.48  ...      -1.389185       -7.878462

[5 rows x 28 columns]
['-SN' 'SN' '-RA' 'RA' '-FZRA' '+SN' '+RA' '+TSRA' 'TSRA' '-TSRA']
7126


2011

In [0]:
all_files2011 = [] 
for year in range(2011, 2012):
    folder, files = folderNames(year)
    all_files2011.append(files) # creates a list of lists 

for x in all_files2011:
    del x[0]
print(all_files2011)

In [0]:
df2011 = PrecipRows('64010KGRB201101.dat')
for list in all_files2011:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2011 = df2011.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2011.head())
print(df2011.current_wx1.unique())
print(len(df2011))

df2011.to_csv('df2011.csv', index=False)

2012

In [0]:
all_files2012 = [] 
for year in range(2012, 2013):
    folder, files = folderNames(year)
    all_files2012.append(files) # creates a list of lists 

for x in all_files2012:
    del x[0]
print(all_files2012)

[['64010KGRB201202.dat', '64010KGRB201203.dat', '64010KGRB201204.dat', '64010KGRB201205.dat', '64010KGRB201206.dat', '64010KGRB201207.dat', '64010KGRB201208.dat', '64010KGRB201209.dat', '64010KGRB201210.dat', '64010KGRB201211.dat', '64010KGRB201212.dat']]


In [0]:
df2012 = PrecipRows('64010KGRB201201.dat')
for list in all_files2012:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2012 = df2012.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2012.head())
print(df2012.current_wx1.unique())
print(len(df2012))

df2012.to_csv('df2012.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...       0.000000        0.000000
1       KGRB     44.48  ...       0.000000        0.000000
2       KGRB     44.48  ...       7.517541       -2.736161
3       KGRB     44.48  ...       9.848078       -1.736482
4       KGRB     44.48  ...      11.276311       -4.104242

[5 rows x 28 columns]
['-RA' '-SN' 'SN' '-FZRA' 'RA' '+SN' '+RA' '-TSRA' 'TSRA' '+TSRA']
6284


2013

In [0]:
all_files2013 = [] 
for year in range(2013, 2014):
    folder, files = folderNames(year)
    all_files2013.append(files) # creates a list of lists 

for x in all_files2013:
    del x[0]
print(all_files2013)

In [0]:
df2013 = PrecipRows('64010KGRB201301.dat')
for list in all_files2013:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2013 = df2013.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2013.head())
print(df2013.current_wx1.unique())
print(len(df2013))

df2013.to_csv('df2013.csv', index=False)

2014

In [0]:
all_files2014 = [] 
for year in range(2014, 2015):
    folder, files = folderNames(year)
    all_files2014.append(files) # creates a list of lists 

for x in all_files2014:
    del x[0]
print(all_files2014)

[['64010KGRB201402.dat', '64010KGRB201403.dat', '64010KGRB201404.dat', '64010KGRB201405.dat', '64010KGRB201406.dat', '64010KGRB201407.dat', '64010KGRB201408.dat', '64010KGRB201409.dat', '64010KGRB201410.dat', '64010KGRB201411.dat', '64010KGRB201412.dat']]


In [0]:
df2014 = PrecipRows('64010KGRB201401.dat')
for list in all_files2014:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2014 = df2014.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2014.head())
print(df2014.current_wx1.unique())
print(len(df2014))

df2014.to_csv('df2014.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...      -1.736482       -9.848078
1       KGRB     44.48  ...      -1.910130      -10.832885
2       KGRB     44.48  ...      -3.420201       -9.396926
3       KGRB     44.48  ...      -2.257426      -12.802501
4       KGRB     44.48  ...      -3.762222      -10.336619

[5 rows x 28 columns]
['-SN' '-RA' 'RA' 'SN' '+SN' '-FZRA' 'TSRA' '-TSRA' '+RA' 'FZRA' '+TSRA']
8582


2015

In [0]:
all_files2015 = [] 
for year in range(2015, 2016):
    folder, files = folderNames(year)
    all_files2015.append(files) # creates a list of lists 

for x in all_files2015:
    del x[0]
print(all_files2015)

In [0]:
df2015 = PrecipRows('64010KGRB201501.dat')
for list in all_files2015:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2015 = df2015.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2015.head())
print(df2015.current_wx1.unique())
print(len(df2015))

df2015.to_csv('df2015.csv', index=False)

2016

In [0]:
all_files2016 = [] 
for year in range(2016, 2017):
    folder, files = folderNames(year)
    all_files2016.append(files) # creates a list of lists 

for x in all_files2016:
    del x[0]
print(all_files2016)

[['64010KGRB201602.dat', '64010KGRB201603.dat', '64010KGRB201604.dat', '64010KGRB201605.dat', '64010KGRB201606.dat', '64010KGRB201607.dat', '64010KGRB201608.dat', '64010KGRB201609.dat', '64010KGRB201610.dat', '64010KGRB201611.dat', '64010KGRB201612.dat']]


In [0]:
df2016 = PrecipRows('64010KGRB201601.dat')
for list in all_files2016:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2016 = df2016.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2016.head())
print(df2016.current_wx1.unique())
print(len(df2016))

df2016.to_csv('df2016.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


done
  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...       7.794229    4.500000e+00
1       KGRB     44.48  ...      10.336619    3.762222e+00
2       KGRB     44.48  ...      10.832885    1.910130e+00
3       KGRB     44.48  ...      12.802501    2.257426e+00
4       KGRB     44.48  ...      13.000000    2.388061e-15

[5 rows x 28 columns]
['-SN' '-RA' 'RA' '-FZRA' 'SN' '+SN' '-TSRA' 'TSRA' '+TSRA' '+RA']
8448


2017

In [0]:
all_files2017 = [] 
for year in range(2017, 2018):
    folder, files = folderNames(year)
    all_files2017.append(files) # creates a list of lists 

for x in all_files2017:
    del x[0]
print(all_files2017)

In [0]:
df2017 = PrecipRows('64010KGRB201701.dat')
for list in all_files2017:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2017 = df2017.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2017.head())
print(df2017.current_wx1.unique())
print(len(df2017))

df2017.to_csv('df2017.csv', index=False)

2018

In [0]:
all_files2018 = [] 
for year in range(2018, 2019):
    folder, files = folderNames(year)
    all_files2018.append(files) # creates a list of lists 

for x in all_files2018:
    del x[0]
print(all_files2018)

[['64010KGRB201802.dat', '64010KGRB201803.dat', '64010KGRB201804.dat', '64010KGRB201805.dat', '64010KGRB201806.dat', '64010KGRB201807.dat', '64010KGRB201808.dat', '64010KGRB201809.dat', '64010KGRB201810.dat', '64010KGRB201811.dat', '64010KGRB201812.dat']]


In [0]:
df2018 = PrecipRows('64010KGRB201801.dat')
for list in all_files2018:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2018 = df2018.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2018.head())
print(df2018.current_wx1.unique())
print(len(df2018))

df2018.to_csv('df2018.csv', index=False)

/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


  station_id  latitude  ...  eastward_wind  northward_wind
0       KGRB     44.48  ...      10.392305        6.000000
1       KGRB     44.48  ...      10.724622        8.999027
2       KGRB     44.48  ...      13.856406        8.000000
3       KGRB     44.48  ...       8.426489        7.070664
4       KGRB     44.48  ...       7.713451        9.192533

[5 rows x 28 columns]
['-SN' '-RA' 'RA' '-FZRA' 'FZRA' 'SN' '+RA' '+SN' '-TSRA' '+TSRA' 'TSRA']
7738


2019

In [0]:
all_files2019 = [] 
for year in range(2019, 2020):
    folder, files = folderNames(year)
    all_files2019.append(files) # creates a list of lists 

for x in all_files2019:
    del x[0]
print(all_files2019)

In [0]:
df2019 = PrecipRows('64010KGRB201901.dat')
for list in all_files2019:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2019 = df2019.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2019.head())
print(df2019.current_wx1.unique())
print(len(df2019))

df2019.to_csv('df2019.csv', index=False)

2020

In [0]:
all_files2020 = [] 
for year in range(2020, 2021):
    folder, files = folderNames(year)
    all_files2020.append(files) # creates a list of lists 

for x in all_files2020:
    del x[3:12]
    del x[0]
print(all_files2020)

In [0]:
df2020 = PrecipRows('64010KGRB202001.dat')
for list in all_files2020:
    for file in list: 
        try:
            df_precip = PrecipRows(file)
            df2020 = df2020.append(df_precip, ignore_index=True, sort=False)
        except: 
            print("done")

print(df2020.head())
print(df2020.current_wx1.unique())
print(len(df2020))

df2020.to_csv('df2020.csv', index=False)

What Some of the Codes Mean 

In [0]:
# Current Wx
# -SN = light snow
# -RA = light rain 
# RA = heavy rain 
# SN = heavy snow 
# BR = mist 
# FG = fog
# HZ = haze 
# FZFG = freezing fog 
#-FZRA = light freezing rain 

# Cloud Types
# OVC = unbroken cloud layer (overcast)
# BKN = over half the sky cloud layer 
# SCT = scattered clouds 
# FEW = few clouds 
# CLR = no clouds 
# VV = vertical visibility (clouds cannot be seen because of fog or heavy precipitation)

array(['-SN', 'BR', '-RA', 'RA', 'FG', 'SN', 'HZ', 'FZFG', '-FZRA'],
      dtype=object)

# Combining Dataframes

In [0]:
# Import packages
# https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/
import os 
import glob 

In [0]:
def csvNames():
    csv_prefix = 'df'
    csv_suffix = '.csv'
    years = []
    for year in range(2000, 2021):
        years.append(year)
    csv_names = [csv_prefix + str(year) + csv_suffix for year in years]
    return csv_names

csv_names = csvNames()

In [0]:
combined_csv = pd.concat([pd.read_csv(csv) for csv in csv_names], ignore_index=True)

In [0]:
combined_csv.to_csv('KGRB_precip.csv', index=False)